In [34]:
from datetime import datetime
import hashlib
class Block:

    def __init__(self,index,previous_hash,transactions,proof,timestamp=None):
        self.index=index
        self.previous_hash=previous_hash
        self.transactions=transactions
        self.proof=proof
        self.timestamp=timestamp
        self.timestamp=datetime.now()
        self.hash=self.calculate_hash()

    def calculate_hash(self):
        block_string= f"{self.index}{self.previous_hash}{self.transactions}{self.proof}"
        return hashlib.sha256(block_string.encode()).hexdigest()

In [35]:
class Blockchain():
    def __init__(self):
        self.chain= []
        self.create_genesis_block()

    def create_genesis_block(self):
        genesis_block=Block(index=0,previous_hash="0",transactions=[],proof=100)
        self.chain.append(genesis_block)

    def add_block(self,transactions,proof):
        previous_block=self.chain[-1]
        new_block=Block(index=len(self.chain),previous_hash=previous_block.hash,transactions=transactions,proof=proof)
        self.chain.append(new_block)


    def is_chain_valid(self):
        for i in range(len(self.chain)):
            current_block=self.chain[i]
            previous_block=self.chain[i-1]

            if current_block.hash != current_block.calculate_hash():
                print(f"Block {i} is compromis")
                return False
            
            if current_block.previous_hash != previous_block.hash:
                print(f"block {i-1} is compromis")
                return False
        return True
        


In [37]:
blockchain=Blockchain()
blockchain.add_block(transactions="alice paie 1 BTC à BOB",proof=12345)
blockchain.add_block(transactions="BOB paie 0.5 BTC à charlie",proof=67800)
blockchain.add_block(transactions="alice paie 2.9 BTC à Slash",proof=45633)

if blockchain.is_chain_valid():
    print("block valid ")
else:
    print("bloc is compromis")

for block in blockchain.chain:
    #print(f"TIME{block.timestamp}: ")
    print(f"Block {block.index}: ")
    print(f"Transactions {block.transactions}: ")
    print(f"Hash {block.hash}: ")
    print(f"Previous Hash {block.previous_hash}: ")
    print()

block -1 is compromis
bloc is compromis
Block 0: 
Transactions []: 
Hash 498b4180973c8b907c1ca00a90c8e11b1f1c82043a86bfb33f13af158558da9c: 
Previous Hash 0: 

Block 1: 
Transactions alice paie 1 BTC à BOB: 
Hash fe675f697dac620706c975b3ccfa4e5bc1f461a9ed1f9c2da9f4df29f068ecfc: 
Previous Hash 498b4180973c8b907c1ca00a90c8e11b1f1c82043a86bfb33f13af158558da9c: 

Block 2: 
Transactions BOB paie 0.5 BTC à charlie: 
Hash af9f745ee2024452ec9ecc755277f7eaf485d0211990c90ea1403874c479162d: 
Previous Hash fe675f697dac620706c975b3ccfa4e5bc1f461a9ed1f9c2da9f4df29f068ecfc: 

Block 3: 
Transactions alice paie 2.9 BTC à Slash: 
Hash 47f092588f24a8013720e6d1c831be3d135ba24b17fc23e3f4fc65693814930c: 
Previous Hash af9f745ee2024452ec9ecc755277f7eaf485d0211990c90ea1403874c479162d: 



TP & EXERCICES

In [22]:
import hashlib
import time

class Bloc:
    def __init__(self, index, horodatage, donnees, hachage_precedent, nonce=0):
        self.index = index
        self.horodatage = time.time()
        self.donnees = donnees
        self.hachage_precedent = hachage_precedent
        self.nonce = nonce
        self.hachage = self.calculer_hachage()
    
    def calculer_hachage(self):
        contenu = f"{self.index}{self.horodatage}{self.donnees}{self.hachage_precedent}{self.nonce}"
        return hashlib.sha256(contenu.encode()).hexdigest()

    def mine_bloc(self, difficulte):
        # Preuve de travail : trouver un hachage commençant par un certain nombre de zéros
        objectif = "0" * difficulte
        while not self.hachage.startswith(objectif):
            self.nonce += 1
            self.hachage = self.calculer_hachage()


In [26]:
class Blockchain:
    def __init__(self, difficulte=4):
        self.chaîne = [self.creer_bloc_genese()]
        self.transactions_en_attente = []
        self.difficulte = difficulte
    
    def creer_bloc_genese(self):
        return Bloc(0, time.time(), "Bloc de genèse", "0")
    
    def obtenir_dernier_bloc(self):
        return self.chaîne[-1]
    
    def ajouter_transaction(self, transaction):
        self.transactions_en_attente.append(transaction)
    
    def ajouter_bloc(self, nouveau_bloc):
        nouveau_bloc.hachage_precedent = self.obtenir_dernier_bloc().hachage
        nouveau_bloc.mine_bloc(self.difficulte)
        self.chaîne.append(nouveau_bloc)
    
    def miner_bloc(self):
        if not self.transactions_en_attente:
            return "Aucune transaction en attente à miner."
        
        nouveau_bloc = Bloc(
            index=len(self.chaîne),
            horodatage=time.time(),
            donnees=self.transactions_en_attente,
            hachage_precedent=self.obtenir_dernier_bloc().hachage
        )
        self.ajouter_bloc(nouveau_bloc)
        self.transactions_en_attente = []
        return nouveau_bloc
    
    def verifier_intégrite(self):
        for i in range(1, len(self.chaîne)):
            bloc_actuel = self.chaîne[i]
            bloc_precedent = self.chaîne[i - 1]
            if bloc_actuel.hachage != bloc_actuel.calculer_hachage():
                return False
            if bloc_actuel.hachage_precedent != bloc_precedent.hachage:
                return False
        return True


In [27]:
class Node:
    def __init__(self):
        self.blockchain = Blockchain()
        self.nœuds_connectes = []
    
    def se_connecter_nœud(self, autre_nœud):
        if autre_nœud not in self.nœuds_connectes:
            self.nœuds_connectes.append(autre_nœud)
            autre_nœud.se_connecter_nœud(self)
    
    def diffuser_chaîne(self):
        for nœud in self.nœuds_connectes:
            nœud.recevoir_chaîne(self.blockchain)
    
    def recevoir_chaîne(self, nouvelle_blockchain):
        if len(nouvelle_blockchain.chaîne) > len(self.blockchain.chaîne) and nouvelle_blockchain.verifier_intégrite():
            self.blockchain = nouvelle_blockchain
    
    def miner_et_diffuser(self):
        nouveau_bloc = self.blockchain.miner_bloc()
        if nouveau_bloc:
            self.diffuser_chaîne()


In [28]:
# Créer deux nœuds du réseau
nœud_1 = Node()
nœud_2 = Node()

# Connecter les nœuds entre eux
nœud_1.se_connecter_nœud(nœud_2)

# Ajouter des transactions et miner sur le nœud 1
nœud_1.blockchain.ajouter_transaction("Alice paie Bob 10 BTC")
nœud_1.blockchain.ajouter_transaction("Bob paie Charlie 5 BTC")
nœud_1.miner_et_diffuser()

# Vérifier l'état de la blockchain du nœud 2
for bloc in nœud_2.blockchain.chaîne:
    print(f"Index: {bloc.index}, Données: {bloc.donnees}, Hachage: {bloc.hachage}")


Index: 0, Données: Bloc de genèse, Hachage: 89758a914d660f718a8f78d3450553ea7e9741eb67ce616cc989ec05bfa3daa7
Index: 1, Données: ['Alice paie Bob 10 BTC', 'Bob paie Charlie 5 BTC'], Hachage: 0000ac064aa858b39c92775ce4690ecba55423347e55a41f522fc3948214e204
